In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
!pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   -- ------------------------------------- 1.0/18.7 MB 7.8 MB/s eta 0:00:03
   ----- ---------------------------------- 2.6/18.7 MB 7.0 MB/s eta 0:00:03
   -------- ------------------------------- 4.2/18.7 MB 7.3 MB/s eta 0:00:02
   ------------ --------------------------- 6.0/18.7 MB 7.9 MB/s eta 0:00:02
   ----------------- ---------------------- 8.1/18.7 MB 8.3 MB/s eta 0:00:02
   --------------------- ------------------ 10.2/18.7 MB 8.6 MB/s eta 0:00:01
   ------------------------- -------------- 11.8/18.7 MB 8.5 MB/s eta 0:00:01
   ------------------------------ --------- 14.2/18.7 MB 8.9 MB/s eta 0:00:01
   ---------------------------------- ----- 16.0/18.7 MB 8.9 MB/s eta 0:00:01
   -------------------------------------- - 18.1/18.7 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 18.7/18.7 MB 8.7 MB/s  0:00:02


In [3]:
path = "january_wl_1994-2024-compressed.pdf"

In [4]:
import fitz  # PyMuPDF
import pandas as pd
import sys
import os

# --- 1. Configuration ---
pdf_path = path
good_csv_path = "good_data.csv"
error_csv_path = "error_data.csv"

# The 8-column header
csv_header = "STATE_UT,DISTRICT,BLOCK,VILLAGE,LATITUDE,LONGITUDE,Date,WL(mbgl)"

# The "anchor" list, as you suggested.
# This is our set of "sentinel" values.
STATES_AND_UTS = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
    'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka',
    'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
    'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim',
    'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand',
    'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh',
    'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Jammu and Kashmir',
    'Ladakh', 'Lakshadweep', 'Puducherry'
}

# Junk lines to filter out
junk_lines = {
    'STATE_UT', 'DISTRICT', 'BLOCK', 'VILLAGE', 'LATITUDE', 'LONGITUDE',
    'Date', 'WL(mbgl)',
    'January month Depth to Water Level (In mbgl) Data of Unconfined Aquifer',
    ''
}

# --- 2. Main Extraction Loop (Anchor Logic) ---
print(f"Opening PDF with PyMuPDF (fitz): {pdf_path}...")
doc = None
current_row = []  # This will store the cells for one row
good_rows_written = 0
bad_rows_written = 0

try:
    doc = fitz.open(pdf_path)
    total_pages = doc.page_count
    print(f"Total pages found: {total_pages}")

    # Open BOTH output files
    with open(good_csv_path, 'w', encoding='utf-8') as f_good, \
         open(error_csv_path, 'w', encoding='utf-8') as f_error:

        # Write the header to both files
        f_good.write(csv_header + '\n')
        f_error.write(csv_header + '\n')
        print(f"Output files '{good_csv_path}' and '{error_csv_path}' created.")

        # Loop through every page
        for i in range(total_pages):

            if (i + 1) % 500 == 0 or (i + 1) == total_pages:
                print(f"Processing page {i + 1} of {total_pages}...")

            page = doc.load_page(i)
            text = page.get_text("text")

            if not text:
                continue

            lines = text.split('\n')

            for line in lines:
                cleaned_line = line.strip()

                # 1. Skip all known junk
                if not cleaned_line or cleaned_line in junk_lines:
                    continue

                # 2. Check if the line is an ANCHOR
                if cleaned_line in STATES_AND_UTS:
                    # This is the start of a new row.
                    # First, we must handle the 'current_row' we were building.

                    if current_row: # If it's not empty
                        # Check if the row we *were* building is complete
                        if len(current_row) == 8:
                            f_good.write(','.join(current_row) + '\n')
                            good_rows_written += 1
                        else:
                            # It's an incomplete/corrupt row
                            f_error.write(','.join(current_row) + '\n')
                            bad_rows_written += 1

                    # 3. Start the NEW row, anchored by the state
                    current_row = [cleaned_line]

                # 4. If it's NOT an anchor, add it to the current row
                else:
                    # Only add if we've already found an anchor
                    if current_row:
                        current_row.append(cleaned_line)

                        # Optimization: If we just added the 8th item,
                        # write it out immediately and clear the row.
                        # This handles perfectly formatted rows quickly.
                        if len(current_row) == 8:
                            f_good.write(','.join(current_row) + '\n')
                            good_rows_written += 1
                            current_row = [] # Reset for the next anchor
                    # else:
                        # This is junk data appearing *before* the first
                        # state anchor on a page. We ignore it.

    # --- End of Loop ---
    # Handle the very last row left in the buffer
    if current_row:
        if len(current_row) == 8:
            f_good.write(','.join(current_row) + '\n')
            good_rows_written += 1
        else:
            f_error.write(','.join(current_row) + '\n')
            bad_rows_written += 1

    print("\n--- Extraction Complete ---")
    print(f"Good rows written: {good_rows_written}")
    print(f"Bad/partial rows written: {bad_rows_written}")

    good_size = os.path.getsize(good_csv_path) / (1024*1024)
    error_size = os.path.getsize(error_csv_path) / (1024*1024)

    print(f"Size of '{good_csv_path}': {good_size:.2f} MB")
    print(f"Size of '{error_csv_path}': {error_size:.2f} MB")

except FileNotFoundError:
    print(f"Error: The file was not found at {pdf_path}")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
finally:
    if doc:
        doc.close()
    print("PDF document closed.")

# --- 3. Load the GOOD data for analysis ---
if good_rows_written > 0:
    print(f"\nLoading good data from '{good_csv_path}' into DataFrame...")

    try:
        # We still use chunks, as the good file might be huge
        chunk_iterator = pd.read_csv(good_csv_path, chunksize=1000000, on_bad_lines='skip')

        list_of_dataframes = []
        for i, chunk in enumerate(chunk_iterator):
            print(f"Processing data chunk {i+1}...")

            if chunk.empty:
                continue

            # Convert types (no more column hacks needed)
            chunk['WL(mbgl)'] = pd.to_numeric(chunk['WL(mbgl)'], errors='coerce')
            chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce') # Flexible date parsing
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

            list_of_dataframes.append(chunk)

        if list_of_dataframes:
            print("Combining all processed chunks into one DataFrame...")
            df = pd.concat(list_of_dataframes)

            print("\n--- DataFrame is Ready for Analysis ---")
            print(df.head())
            print("\nDataFrame Info:")
            df.info()
        else:
            print("No valid data chunks were loaded.")

    except Exception as e:
        print(f"An error occurred while loading the CSV: {e}")
else:
    print("\nNo good rows were extracted. The DataFrame is empty.")
    print(f"Please check '{error_csv_path}' to see what data was found.")

Opening PDF with PyMuPDF (fitz): january_wl_1994-2024-compressed.pdf...
Total pages found: 10242
Output files 'good_data.csv' and 'error_data.csv' created.
Processing page 500 of 10242...
Processing page 1000 of 10242...
Processing page 1500 of 10242...
Processing page 2000 of 10242...
Processing page 2500 of 10242...
Processing page 3000 of 10242...
Processing page 3500 of 10242...
Processing page 4000 of 10242...
Processing page 4500 of 10242...
Processing page 5000 of 10242...
Processing page 5500 of 10242...
Processing page 6000 of 10242...
Processing page 6500 of 10242...
Processing page 7000 of 10242...
Processing page 7500 of 10242...
Processing page 8000 of 10242...
Processing page 8500 of 10242...
Processing page 9000 of 10242...
Processing page 9500 of 10242...
Processing page 10000 of 10242...
Processing page 10242 of 10242...

--- Extraction Complete ---
Good rows written: 383226
Bad/partial rows written: 5866
Size of 'good_data.csv': 25.88 MB
Size of 'error_data.csv': 0.45

C:\Users\osadm\AppData\Local\Temp\ipykernel_3644\1915151008.py:158: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce') # Flexible date parsing


In [5]:
import fitz  # PyMuPDF
import pandas as pd
import sys
import os
import re  # <-- We need this for date validation

# --- 1. Helper Validation Functions ---

def is_date(s):
    """Checks if a string loosely looks like a date (e.g., 10-04-24 or 2024-01-01)"""
    # This regex checks for dd-mm-yy, dd-mm-yyyy, yyyy-mm-dd etc.
    if re.match(r'^\d{2}-\d{2}-\d{2,4}$', s) or re.match(r'^\d{4}-\d{2}-\d{2}$', s):
        return True
    return False

def is_number(s):
    """Checks if a string can be converted to a float."""
    try:
        float(s)
        return True
    except (ValueError, TypeError):
        return False

# --- 2. Configuration ---
pdf_path = path
good_csv_path = "good_data_v3.csv"
error_csv_path = "error_data_v3.csv"

csv_header = "STATE_UT,DISTRICT,BLOCK,VILLAGE,LATITUDE,LONGITUDE,Date,WL(mbgl)"

STATES_AND_UTS = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
    'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka',
    'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
    'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim',
    'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand',
    'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh',
    'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Jammu and Kashmir',
    'Ladakh', 'Lakshadweep', 'Puducherry'
}

junk_lines = {
    'STATE_UT', 'DISTRICT', 'BLOCK', 'VILLAGE', 'LATITUDE', 'LONGITUDE',
    'Date', 'WL(mbgl)',
    'January month Depth to Water Level (In mbgl) Data of Unconfined Aquifer',
    ''
}

# --- 3. Main Extraction Loop (With Validation) ---
print(f"Opening PDF with PyMuPDF (fitz): {pdf_path}...")
doc = None
good_rows_written = 0
bad_rows_written = 0
child_rows_fixed = 0

last_known_location = []  # Memory: [STATE, ... , LONGITUDE]
temp_buffer = []          # Collects lines

try:
    doc = fitz.open(pdf_path)
    total_pages = doc.page_count
    print(f"Total pages found: {total_pages}")

    with open(good_csv_path, 'w', encoding='utf-8') as f_good, \
         open(error_csv_path, 'w', encoding='utf-8') as f_error:

        f_good.write(csv_header + '\n')
        f_error.write("REASON,DATA\n")
        print(f"Output files '{good_csv_path}' and '{error_csv_path}' created.")

        for i in range(total_pages):

            if (i + 1) % 500 == 0 or (i + 1) == total_pages:
                print(f"Processing page {i + 1} of {total_pages}...")

            page = doc.load_page(i)
            text = page.get_text("text")

            if not text:
                continue

            lines = text.split('\n')

            for line in lines:
                cleaned_line = line.strip()

                if not cleaned_line or cleaned_line in junk_lines:
                    continue

                temp_buffer.append(cleaned_line)

                # --- DECISION LOGIC ---

                # Case 1: Buffer starts with a State Anchor
                if temp_buffer[0] in STATES_AND_UTS:
                    if len(temp_buffer) == 8:
                        # Full 8-item parent row.
                        f_good.write(','.join(temp_buffer) + '\n')
                        last_known_location = temp_buffer[:6] # Save location
                        temp_buffer = [] # Clear buffer
                        good_rows_written += 1
                    elif len(temp_buffer) > 8:
                        # Error: Row is too long
                        f_error.write(f"OVERFLOW_PARENT_ROW,{','.join(temp_buffer)}\n")
                        temp_buffer = []
                        last_known_location = [] # Reset memory, it's corrupt
                        bad_rows_written += 1

                # Case 2: Buffer does NOT start with a state
                else:
                    # We can't do anything without memory
                    if not last_known_location:
                        f_error.write(f"NO_MEMORY,{','.join(temp_buffer)}\n")
                        temp_buffer = []
                        bad_rows_written += 1
                        continue # Move to next line

                    # We have memory, now check the buffer length
                    if len(temp_buffer) == 2:
                        # --- THIS IS THE NEW CHECK ---
                        if is_date(temp_buffer[0]) and is_number(temp_buffer[1]):
                            # It's a valid child row!
                            full_row = last_known_location + temp_buffer
                            f_good.write(','.join(full_row) + '\n')
                            child_rows_fixed += 1
                        else:
                            # It's 2 items, but NOT (Date, WL)
                            # This is the 'Kishorinagar, 10-04-24' case
                            f_error.write(f"INVALID_CHILD_ROW,{','.join(temp_buffer)}\n")
                            bad_rows_written += 1

                        temp_buffer = [] # Clear buffer in both cases

                    elif len(temp_buffer) > 2:
                        # Error: Child row is too long
                        f_error.write(f"OVERFLOW_CHILD_ROW,{','.join(temp_buffer)}\n")
                        temp_buffer = []
                        bad_rows_written += 1

    # --- End of Loop ---
    if temp_buffer:
        f_error.write(f"PARTIAL_END_OF_FILE,{','.join(temp_buffer)}\n")
        bad_rows_written += 1

    print("\n--- Extraction Complete ---")
    print(f"Good 'parent' rows written: {good_rows_written}")
    print(f"Good 'child' rows fixed: {child_rows_fixed} (Thanks to new validation!)")
    print(f"Total good rows: {good_rows_written + child_rows_fixed}")
    print(f"Bad/partial rows logged: {bad_rows_written}")

except Exception as e:
    print(f"An error occurred during extraction: {e}")
finally:
    if doc:
        doc.close()
    print("PDF document closed.")

# --- 4. Load the GOOD data for analysis ---
if (good_rows_written + child_rows_fixed) > 0:
    print(f"\nLoading good data from '{good_csv_path}' into DataFrame...")

    try:
        chunk_iterator = pd.read_csv(good_csv_path, chunksize=1000000, on_bad_lines='skip')
        list_of_dataframes = []

        for i, chunk in enumerate(chunk_iterator):
            print(f"Processing data chunk {i+1}...")
            if chunk.empty: continue

            chunk['WL(mbgl)'] = pd.to_numeric(chunk['WL(mbgl)'], errors='coerce')
            chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

            list_of_dataframes.append(chunk)

        if list_of_dataframes:
            print("Combining all processed chunks into one DataFrame...")
            df = pd.concat(list_of_dataframes)

            print("\n--- DataFrame is Ready for Analysis ---")
            print(df.head())
            print("\nDataFrame Info:")
            df.info()

            # --- Your Pivot Idea ---
            print("\n--- Example of your 'Pivot' idea ---")
            df_pivoted = df.pivot_table(
                index=['STATE_UT', 'DISTRICT', 'BLOCK', 'VILLAGE', 'LATITUDE', 'LONGITUDE'],
                columns='Date',
                values='WL(mbgl)',
                aggfunc='last'
            )
            print(df_pivoted.head())

    except Exception as e:
        print(f"An error occurred while loading the CSV: {e}")
else:
    print("\nNo good rows were extracted. The DataFrame is empty.")

Opening PDF with PyMuPDF (fitz): january_wl_1994-2024-compressed.pdf...
Total pages found: 10242
Output files 'good_data_v3.csv' and 'error_data_v3.csv' created.
Processing page 500 of 10242...
Processing page 1000 of 10242...
Processing page 1500 of 10242...
Processing page 2000 of 10242...
Processing page 2500 of 10242...
Processing page 3000 of 10242...
Processing page 3500 of 10242...
Processing page 4000 of 10242...
Processing page 4500 of 10242...
Processing page 5000 of 10242...
Processing page 5500 of 10242...
Processing page 6000 of 10242...
Processing page 6500 of 10242...
Processing page 7000 of 10242...
Processing page 7500 of 10242...
Processing page 8000 of 10242...
Processing page 8500 of 10242...
Processing page 9000 of 10242...
Processing page 9500 of 10242...
Processing page 10000 of 10242...
Processing page 10242 of 10242...

--- Extraction Complete ---
Good 'parent' rows written: 195635
Good 'child' rows fixed: 2761 (Thanks to new validation!)
Total good rows: 19839

C:\Users\osadm\AppData\Local\Temp\ipykernel_3644\2810973768.py:166: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for i, chunk in enumerate(chunk_iterator):
C:\Users\osadm\AppData\Local\Temp\ipykernel_3644\2810973768.py:171: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')


Processing data chunk 1...
Combining all processed chunks into one DataFrame...

--- DataFrame is Ready for Analysis ---
         STATE_UT               DISTRICT        BLOCK             VILLAGE  \
0  Andhra Pradesh  Alluri Sitharama Raju  Addateegala         Addateegala   
1  Andhra Pradesh  Alluri Sitharama Raju  Addateegala  Mallavaram Mammilu   
2  Andhra Pradesh  Alluri Sitharama Raju  Addateegala           Rayapalli   
3  Andhra Pradesh  Alluri Sitharama Raju  Addateegala     Veerbhadrapuram   
4  Andhra Pradesh  Alluri Sitharama Raju  Ananthagiri          Anantagiri   

   LATITUDE  LONGITUDE       Date  WL(mbgl)  
0  17.46330    82.0271 2024-01-01       9.0  
1  17.50320    82.0124 2024-01-01       4.0  
2  17.57208    82.0085 2024-01-01       1.9  
3  17.43790    82.0740 2024-01-01       1.4  
4  18.23880    83.0011 2024-01-01       5.3  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198368 entries, 0 to 198367
Data columns (total 8 columns):
 #   Column  

In [12]:
import pandas as pd
import sys

# --- This is the correct way to load your CSV ---

good_csv_path = 'good_data.csv'
list_of_dataframes = []
all_chunks_processed = False

print(f"\nLoading good data from '{good_csv_path}' into DataFrame using chunks...")
chunk_iterator = None

try:
    # We use chunksize AND on_bad_lines='skip'
    # 'skip' will automatically ignore line 25,432 (and any other errors)
    chunk_iterator = pd.read_csv(
        good_csv_path,
        chunksize=1000000,
        on_bad_lines='skip' # This is the fix
    )
    all_chunks_processed = True

except FileNotFoundError:
    print(f"Error: The file {good_csv_path} was not found.")
    sys.exit()
except pd.errors.EmptyDataError:
    print("Error: The CSV file is empty.")
    sys.exit()

if all_chunks_processed:
    for i, chunk in enumerate(chunk_iterator):
        print(f"Processing data chunk {i+1}...")
        if chunk.empty:
            print("Chunk is empty, skipping.")
            continue

        # Ensure the columns are what we expect, in case of a bad first chunk
        if list(chunk.columns) != ['STATE_UT', 'DISTRICT', 'BLOCK', 'VILLAGE', 'LATITUDE', 'LONGITUDE', 'Date', 'WL(mbgl)']:
             print(f"Skipping chunk {i+1} due to malformed header.")
             continue

        # Convert types
        chunk['WL(mbgl)'] = pd.to_numeric(chunk['WL(mbgl)'], errors='coerce')
        chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')
        chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
        chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')

        list_of_dataframes.append(chunk)

# --- 4. Combine all chunks into the final DataFrame ---
if list_of_dataframes:
    print("Combining all processed chunks into one DataFrame...")
    df2 = pd.concat(list_of_dataframes)

    print("\n--- DataFrame is Ready for Analysis ---")
    print(df2.head())
    print("\nDataFrame Info:")
    df2.info()
else:
    print("No valid data chunks were loaded. The DataFrame is empty.")


Loading good data from 'good_data.csv' into DataFrame using chunks...
Processing data chunk 1...
Combining all processed chunks into one DataFrame...

--- DataFrame is Ready for Analysis ---
         STATE_UT               DISTRICT        BLOCK             VILLAGE  \
0  Andhra Pradesh  Alluri Sitharama Raju  Addateegala         Addateegala   
1  Andhra Pradesh  Alluri Sitharama Raju  Addateegala  Mallavaram Mammilu   
2  Andhra Pradesh  Alluri Sitharama Raju  Addateegala           Rayapalli   
3  Andhra Pradesh  Alluri Sitharama Raju  Addateegala     Veerbhadrapuram   
4  Andhra Pradesh  Alluri Sitharama Raju  Ananthagiri          Anantagiri   

   LATITUDE  LONGITUDE       Date  WL(mbgl)  
0  17.46330    82.0271 2024-01-01       9.0  
1  17.50320    82.0124 2024-01-01       4.0  
2  17.57208    82.0085 2024-01-01       1.9  
3  17.43790    82.0740 2024-01-01       1.4  
4  18.23880    83.0011 2024-01-01       5.3  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38

C:\Users\osadm\AppData\Local\Temp\ipykernel_3644\2563798867.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')


In [9]:
df.sample(15)

,STATE_UT,DISTRICT,BLOCK,VILLAGE,LATITUDE,LONGITUDE,Date,WL(mbgl)
147979,Uttar Pradesh,Fatehpur,Khajuha,Sarain Bakewar,26.11944,80.48333,2003-01-01,2.1
35611,Andhra Pradesh,Y.S.R.,Kamalapuram,Chadipiralla,14.58752,78.63683,2020-01-01,2.9
144777,Chhattisgarh,Mahasamund,Mahasamund,Mahasamund,21.10833,82.09583,2004-10-01,6.2
93355,Karnataka,Koppal,Kukunoor,Itagi,15.44360,75.96860,2014-05-01,7.8
18307,Gujarat,Rajkot,Rajkot,Sardhar,22.14583,70.98889,2023-10-01,2.5
196301,Rajasthan,Hanumangarh,Nohar,Ramsara,29.25000,74.83330,1994-01-01,22.2
134409,Maharashtra,Yavatmal,Ghatanji,Ghatanji,20.13333,78.31667,2006-10-01,6.3
179605,Odisha,Bargarh,Bheden,Lupursinga,21.24028,83.79806,1997-01-01,2.1
175274,Haryana,Kurukshetra,Pehowa,Pehowa,29.98333,76.58056,1997-01-01,12.7
44867,Kerala,Kasaragod,Kanhangad,Pallikkara,12.41450,75.05140,2019-01-01,10.1


In [11]:
df.shape

(198368, 8)

In [13]:
df2.shape

(383167, 8)

In [14]:
import fitz  # PyMuPDF
import pandas as pd
import os
import re

# --- 1. Helper Validation Functions ---

def is_date(s):
    """Relaxed date check: accepts dd-mm-yy, dd-mm-yyyy, d-m-yy, etc."""
    s = s.strip()
    return bool(re.match(r'^\d{1,2}-\d{1,2}-\d{2,4}$', s)) or bool(re.match(r'^\d{4}-\d{1,2}-\d{1,2}$', s))

def is_number(s):
    """Relaxed numeric check (tolerates commas or misplaced dots)."""
    s = s.strip().replace(',', '')
    try:
        float(s)
        return True
    except:
        return False

# --- 2. Configuration ---

pdf_path = path 
good_csv_path = "good_data_v4.csv"
error_csv_path = "error_data_v4.csv"

csv_header = "STATE_UT,DISTRICT,BLOCK,VILLAGE,LATITUDE,LONGITUDE,Date,WL(mbgl)"

STATES_AND_UTS = {
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
    'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka',
    'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
    'Mizoram', 'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim',
    'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh', 'Uttarakhand',
    'West Bengal', 'Andaman and Nicobar Islands', 'Chandigarh',
    'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Jammu and Kashmir',
    'Ladakh', 'Lakshadweep', 'Puducherry'
}

junk_lines = {
    'STATE_UT', 'DISTRICT', 'BLOCK', 'VILLAGE', 'LATITUDE', 'LONGITUDE',
    'Date', 'WL(mbgl)', '',
    'January month Depth to Water Level (In mbgl) Data of Unconfined Aquifer'
}

# --- 3. Main Extraction ---

print(f"Opening PDF: {pdf_path}")
doc = None

good_rows_written = 0
bad_rows_written = 0
child_rows_fixed = 0

last_known_location = []  # [STATE, DISTRICT, BLOCK, VILLAGE, LAT, LONG]
temp_buffer = []

try:
    doc = fitz.open(pdf_path)
    total_pages = doc.page_count
    print(f"Total pages: {total_pages}")

    with open(good_csv_path, 'w', encoding='utf-8') as f_good, \
         open(error_csv_path, 'w', encoding='utf-8') as f_error:

        f_good.write(csv_header + '\n')
        f_error.write("REASON,DATA\n")

        for i in range(total_pages):
            if (i + 1) % 500 == 0 or (i + 1) == total_pages:
                print(f"Processing page {i + 1} / {total_pages}")

            page = doc.load_page(i)
            text = page.get_text("text")

            if not text:
                continue

            lines = [ln.strip() for ln in text.split('\n') if ln.strip()]

            for line in lines:
                if line in junk_lines:
                    continue

                # Auto-fix merged words like "KonaseemaAinavilli"
                line = re.sub(r'([a-z])([A-Z])', r'\1 \2', line)

                temp_buffer.append(line)

                # --- Case 1: starts with state (new parent row)
                if temp_buffer[0] in STATES_AND_UTS:
                    if 7 <= len(temp_buffer) <= 9:
                        f_good.write(','.join(temp_buffer[:8]) + '\n')
                        last_known_location = temp_buffer[:6]
                        temp_buffer = []
                        good_rows_written += 1
                    elif len(temp_buffer) > 9:
                        f_error.write(f"OVERFLOW_PARENT_ROW,{','.join(temp_buffer)}\n")
                        temp_buffer = []
                        bad_rows_written += 1

                # --- Case 2: continuation (child rows)
                else:
                    if last_known_location and len(temp_buffer) == 2:
                        a, b = temp_buffer
                        if is_date(a) and is_number(b):
                            f_good.write(','.join(last_known_location + temp_buffer) + '\n')
                            child_rows_fixed += 1
                        else:
                            f_error.write(f"INVALID_CHILD_ROW,{','.join(temp_buffer)}\n")
                            bad_rows_written += 1
                        temp_buffer = []

                    elif len(temp_buffer) > 2:
                        # possible carry-over or overflow; try auto-correct
                        joined = ' '.join(temp_buffer)
                        tokens = joined.split()
                        if len(tokens) >= 8:
                            f_good.write(','.join(tokens[:8]) + '\n')
                            last_known_location = tokens[:6]
                            good_rows_written += 1
                        else:
                            f_error.write(f"SHORT_ROW,{','.join(tokens)}\n")
                            bad_rows_written += 1
                        temp_buffer = []

    print("\n--- Extraction Complete ---")
    print(f"Good parent rows: {good_rows_written}")
    print(f"Child rows fixed: {child_rows_fixed}")
    print(f"Total good rows: {good_rows_written + child_rows_fixed}")
    print(f"Bad rows: {bad_rows_written}")

except Exception as e:
    print(f"❌ Extraction error: {e}")

finally:
    if doc:
        doc.close()
    print("PDF closed.")

# --- 4. Load and Clean ---

if good_rows_written + child_rows_fixed > 0:
    try:
        print(f"\nLoading {good_csv_path} into DataFrame...")
        df_chunks = []
        for chunk in pd.read_csv(good_csv_path, chunksize=1000000, on_bad_lines='skip'):
            chunk['WL(mbgl)'] = pd.to_numeric(chunk['WL(mbgl)'], errors='coerce')
            chunk['LATITUDE'] = pd.to_numeric(chunk['LATITUDE'], errors='coerce')
            chunk['LONGITUDE'] = pd.to_numeric(chunk['LONGITUDE'], errors='coerce')
            chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')
            df_chunks.append(chunk)

        if df_chunks:
            df = pd.concat(df_chunks)
            print("\n✅ DataFrame Ready")
            print(df.head())
            print(df.info())

    except Exception as e:
        print(f"Error reading CSV: {e}")
else:
    print("No good rows extracted.")


Opening PDF: january_wl_1994-2024-compressed.pdf
Total pages: 10242
Processing page 500 / 10242
Processing page 1000 / 10242
Processing page 1500 / 10242
Processing page 2000 / 10242
Processing page 2500 / 10242
Processing page 3000 / 10242
Processing page 3500 / 10242
Processing page 4000 / 10242
Processing page 4500 / 10242
Processing page 5000 / 10242
Processing page 5500 / 10242
Processing page 6000 / 10242
Processing page 6500 / 10242
Processing page 7000 / 10242
Processing page 7500 / 10242
Processing page 8000 / 10242
Processing page 8500 / 10242
Processing page 9000 / 10242
Processing page 9500 / 10242
Processing page 10000 / 10242
Processing page 10242 / 10242

--- Extraction Complete ---
Good parent rows: 5325
Child rows fixed: 937
Total good rows: 6262
Bad rows: 1534494
PDF closed.

Loading good_data_v4.csv into DataFrame...

✅ DataFrame Ready
         STATE_UT               DISTRICT                          BLOCK  \
0  Andhra Pradesh  Alluri Sitharama Raju                  

C:\Users\osadm\AppData\Local\Temp\ipykernel_3644\2871025819.py:152: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['Date'] = pd.to_datetime(chunk['Date'], errors='coerce')


version 5

In [15]:
import fitz
import pandas as pd
import re

def is_date(tok):
    return bool(re.match(r'^\d{1,2}-\d{1,2}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

pdf_path = path   # your PDF
good_csv = "good_data_v5.csv"
error_csv = "error_data_v5.csv"

STATES = {
    'Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil Nadu','Telangana','Tripura',
    'Uttar Pradesh','Uttarakhand','West Bengal','Andaman and Nicobar Islands',
    'Chandigarh','Dadra and Nagar Haveli and Daman and Diu','Delhi',
    'Jammu and Kashmir','Ladakh','Lakshadweep','Puducherry'
}

doc = fitz.open(pdf_path)
tokens = []

print("Tokenizing PDF ...")
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)   # fix merged words
    tokens += text.split()

rows = []
bad_rows = 0
current = []

print(f"Total tokens: {len(tokens):,}")

for tok in tokens:
    # Start a new row when we hit a state name
    if tok in STATES:
        if len(current) == 8:
            rows.append(current)
        elif current:
            bad_rows += 1
        current = [tok]
    else:
        current.append(tok)
        if len(current) == 8:
            # basic sanity: lat/lon/date/wl
            lat, lon, date, wl = current[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(current)
            else:
                bad_rows += 1
            current = []

# Flush tail
if len(current) == 8:
    rows.append(current)
else:
    bad_rows += 1

print(f"\nGood rows: {len(rows):,}")
print(f"Bad rows: {bad_rows:,}")

# Write to CSV
pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
]).to_csv(good_csv, index=False)

print(f"\n✅ Saved clean data to {good_csv}")


Tokenizing PDF ...
Total tokens: 3,572,403

Good rows: 223,830
Bad rows: 275,753

✅ Saved clean data to good_data_v5.csv


v6

In [16]:
import fitz
import pandas as pd
import re

# --- Helpers ---

def is_date(tok):
    tok = tok.strip()
    # Accept 01-01-24, 01/01/2024, 01.01.24, 1-Jan-24, Jan-24
    return bool(re.match(r'^(\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})$', tok)) \
        or bool(re.match(r'^\d{1,2}-[A-Za-z]{3,}-\d{2,4}$', tok)) \
        or bool(re.match(r'^[A-Za-z]{3,}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

def merge_multiword_states(text):
    """Rejoin known multi-word state names to single tokens."""
    replacements = [
        ("Andaman and Nicobar Islands", "Andaman_Nicobar_Islands"),
        ("Dadra and Nagar Haveli and Daman and Diu", "Dadra_Nagar_Haveli_Daman_Diu"),
        ("Jammu and Kashmir", "Jammu_Kashmir"),
        ("Madhya Pradesh", "Madhya_Pradesh"),
        ("Tamil Nadu", "Tamil_Nadu"),
        ("Uttar Pradesh", "Uttar_Pradesh"),
        ("West Bengal", "West_Bengal"),
        ("Andhra Pradesh", "Andhra_Pradesh"),
    ]
    for a,b in replacements:
        text = text.replace(a, b)
    return text

# --- Config ---
pdf_path = path
good_csv = "good_data_v6.csv"

STATES = {
    'Andhra_Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya_Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil_Nadu','Telangana','Tripura',
    'Uttar_Pradesh','Uttarakhand','West_Bengal',
    'Andaman_Nicobar_Islands','Chandigarh',
    'Dadra_Nagar_Haveli_Daman_Diu','Delhi',
    'Jammu_Kashmir','Ladakh','Lakshadweep','Puducherry'
}

# --- 1. Tokenize ---
doc = fitz.open(pdf_path)
tokens = []
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = merge_multiword_states(text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    tokens += text.split()

print(f"Total tokens: {len(tokens):,}")

# --- 2. Scan tokens with sliding window ---
rows = []
bad = 0
current = []

for idx, tok in enumerate(tokens):
    if tok in STATES:
        # flush old row if valid
        if len(current) == 8:
            rows.append(current)
        current = [tok]
        continue

    if not current:
        continue

    current.append(tok)

    # When buffer is between 8–12 tokens, test every 8-token slice
    if len(current) >= 8:
        found = False
        for start in range(max(0, len(current)-12), len(current)-7):
            slice8 = current[start:start+8]
            lat, lon, date, wl = slice8[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(slice8)
                found = True
                current = []
                break
        if not found and len(current) > 12:
            bad += 1
            current = []

# --- 3. Save ---
df = pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
])
print(f"\nGood rows: {len(df):,} | Bad fragments: {bad:,}")
df.to_csv(good_csv, index=False)
print(f"✅ Saved clean data to {good_csv}")


Total tokens: 3,449,989

Good rows: 383,559 | Bad fragments: 65
✅ Saved clean data to good_data_v6.csv


In [2]:
import fitz
import pandas as pd
import re

# --- Helpers ---

def is_date(tok):
    tok = tok.strip()
    # Accept 01-01-24, 01/01/2024, 01.01.24, 1-Jan-24, Jan-24
    return bool(re.match(r'^(\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})$', tok)) \
        or bool(re.match(r'^\d{1,2}-[A-Za-z]{3,}-\d{2,4}$', tok)) \
        or bool(re.match(r'^[A-Za-z]{3,}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

def merge_multiword_states(text):
    """Rejoin known multi-word state names to single tokens."""
    replacements = [
        ("Andaman and Nicobar Islands", "Andaman_Nicobar_Islands"),
        ("Dadra and Nagar Haveli and Daman and Diu", "Dadra_Nagar_Haveli_Daman_Diu"),
        ("Jammu and Kashmir", "Jammu_Kashmir"),
        ("Madhya Pradesh", "Madhya_Pradesh"),
        ("Tamil Nadu", "Tamil_Nadu"),
        ("Uttar Pradesh", "Uttar_Pradesh"),
        ("West Bengal", "West_Bengal"),
        ("Andhra Pradesh", "Andhra_Pradesh"),
    ]
    for a,b in replacements:
        text = text.replace(a, b)
    return text

# --- Config ---
pdf_path = "august_wl_1994-2023_compressed.pdf"
good_csv = "good_data_v6_aug.csv"

STATES = {
    'Andhra_Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya_Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil_Nadu','Telangana','Tripura',
    'Uttar_Pradesh','Uttarakhand','West_Bengal',
    'Andaman_Nicobar_Islands','Chandigarh',
    'Dadra_Nagar_Haveli_Daman_Diu','Delhi',
    'Jammu_Kashmir','Ladakh','Lakshadweep','Puducherry'
}

# --- 1. Tokenize ---
doc = fitz.open(pdf_path)
tokens = []
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = merge_multiword_states(text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    tokens += text.split()

print(f"Total tokens: {len(tokens):,}")

# --- 2. Scan tokens with sliding window ---
rows = []
bad = 0
current = []

for idx, tok in enumerate(tokens):
    if tok in STATES:
        # flush old row if valid
        if len(current) == 8:
            rows.append(current)
        current = [tok]
        continue

    if not current:
        continue

    current.append(tok)

    # When buffer is between 8–12 tokens, test every 8-token slice
    if len(current) >= 8:
        found = False
        for start in range(max(0, len(current)-12), len(current)-7):
            slice8 = current[start:start+8]
            lat, lon, date, wl = slice8[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(slice8)
                found = True
                current = []
                break
        if not found and len(current) > 12:
            bad += 1
            current = []

# --- 3. Save ---
df = pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
])
print(f"\nGood rows: {len(df):,} | Bad fragments: {bad:,}")
df.to_csv(good_csv, index=False)
print(f"✅ Saved clean data to {good_csv}")


Total tokens: 3,209,238

Good rows: 353,246 | Bad fragments: 55
✅ Saved clean data to good_data_v6_aug.csv


In [3]:
import fitz
import pandas as pd
import re

# --- Helpers ---

def is_date(tok):
    tok = tok.strip()
    # Accept 01-01-24, 01/01/2024, 01.01.24, 1-Jan-24, Jan-24
    return bool(re.match(r'^(\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})$', tok)) \
        or bool(re.match(r'^\d{1,2}-[A-Za-z]{3,}-\d{2,4}$', tok)) \
        or bool(re.match(r'^[A-Za-z]{3,}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

def merge_multiword_states(text):
    """Rejoin known multi-word state names to single tokens."""
    replacements = [
        ("Andaman and Nicobar Islands", "Andaman_Nicobar_Islands"),
        ("Dadra and Nagar Haveli and Daman and Diu", "Dadra_Nagar_Haveli_Daman_Diu"),
        ("Jammu and Kashmir", "Jammu_Kashmir"),
        ("Madhya Pradesh", "Madhya_Pradesh"),
        ("Tamil Nadu", "Tamil_Nadu"),
        ("Uttar Pradesh", "Uttar_Pradesh"),
        ("West Bengal", "West_Bengal"),
        ("Andhra Pradesh", "Andhra_Pradesh"),
    ]
    for a,b in replacements:
        text = text.replace(a, b)
    return text

# --- Config ---
pdf_path = "pre-monsoon_1994-2003.pdf"
good_csv = "good_data_v6_pre1.csv"

STATES = {
    'Andhra_Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya_Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil_Nadu','Telangana','Tripura',
    'Uttar_Pradesh','Uttarakhand','West_Bengal',
    'Andaman_Nicobar_Islands','Chandigarh',
    'Dadra_Nagar_Haveli_Daman_Diu','Delhi',
    'Jammu_Kashmir','Ladakh','Lakshadweep','Puducherry'
}

# --- 1. Tokenize ---
doc = fitz.open(pdf_path)
tokens = []
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = merge_multiword_states(text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    tokens += text.split()

print(f"Total tokens: {len(tokens):,}")

# --- 2. Scan tokens with sliding window ---
rows = []
bad = 0
current = []

for idx, tok in enumerate(tokens):
    if tok in STATES:
        # flush old row if valid
        if len(current) == 8:
            rows.append(current)
        current = [tok]
        continue

    if not current:
        continue

    current.append(tok)

    # When buffer is between 8–12 tokens, test every 8-token slice
    if len(current) >= 8:
        found = False
        for start in range(max(0, len(current)-12), len(current)-7):
            slice8 = current[start:start+8]
            lat, lon, date, wl = slice8[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(slice8)
                found = True
                current = []
                break
        if not found and len(current) > 12:
            bad += 1
            current = []

# --- 3. Save ---
df = pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
])
print(f"\nGood rows: {len(df):,} | Bad fragments: {bad:,}")
df.to_csv(good_csv, index=False)
print(f"✅ Saved clean data to {good_csv}")


Total tokens: 999,114

Good rows: 111,313 | Bad fragments: 23
✅ Saved clean data to good_data_v6_pre1.csv


In [4]:
import fitz
import pandas as pd
import re

# --- Helpers ---

def is_date(tok):
    tok = tok.strip()
    # Accept 01-01-24, 01/01/2024, 01.01.24, 1-Jan-24, Jan-24
    return bool(re.match(r'^(\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})$', tok)) \
        or bool(re.match(r'^\d{1,2}-[A-Za-z]{3,}-\d{2,4}$', tok)) \
        or bool(re.match(r'^[A-Za-z]{3,}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

def merge_multiword_states(text):
    """Rejoin known multi-word state names to single tokens."""
    replacements = [
        ("Andaman and Nicobar Islands", "Andaman_Nicobar_Islands"),
        ("Dadra and Nagar Haveli and Daman and Diu", "Dadra_Nagar_Haveli_Daman_Diu"),
        ("Jammu and Kashmir", "Jammu_Kashmir"),
        ("Madhya Pradesh", "Madhya_Pradesh"),
        ("Tamil Nadu", "Tamil_Nadu"),
        ("Uttar Pradesh", "Uttar_Pradesh"),
        ("West Bengal", "West_Bengal"),
        ("Andhra Pradesh", "Andhra_Pradesh"),
    ]
    for a,b in replacements:
        text = text.replace(a, b)
    return text

# --- Config ---
pdf_path = "pre-monsoon_2004-2013.pdf"
good_csv = "good_data_v6_pre2.csv"

STATES = {
    'Andhra_Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya_Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil_Nadu','Telangana','Tripura',
    'Uttar_Pradesh','Uttarakhand','West_Bengal',
    'Andaman_Nicobar_Islands','Chandigarh',
    'Dadra_Nagar_Haveli_Daman_Diu','Delhi',
    'Jammu_Kashmir','Ladakh','Lakshadweep','Puducherry'
}

# --- 1. Tokenize ---
doc = fitz.open(pdf_path)
tokens = []
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = merge_multiword_states(text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    tokens += text.split()

print(f"Total tokens: {len(tokens):,}")

# --- 2. Scan tokens with sliding window ---
rows = []
bad = 0
current = []

for idx, tok in enumerate(tokens):
    if tok in STATES:
        # flush old row if valid
        if len(current) == 8:
            rows.append(current)
        current = [tok]
        continue

    if not current:
        continue

    current.append(tok)

    # When buffer is between 8–12 tokens, test every 8-token slice
    if len(current) >= 8:
        found = False
        for start in range(max(0, len(current)-12), len(current)-7):
            slice8 = current[start:start+8]
            lat, lon, date, wl = slice8[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(slice8)
                found = True
                current = []
                break
        if not found and len(current) > 12:
            bad += 1
            current = []

# --- 3. Save ---
df = pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
])
print(f"\nGood rows: {len(df):,} | Bad fragments: {bad:,}")
df.to_csv(good_csv, index=False)
print(f"✅ Saved clean data to {good_csv}")


Total tokens: 1,070,014

Good rows: 117,671 | Bad fragments: 21
✅ Saved clean data to good_data_v6_pre2.csv


In [5]:
import fitz
import pandas as pd
import re

# --- Helpers ---

def is_date(tok):
    tok = tok.strip()
    # Accept 01-01-24, 01/01/2024, 01.01.24, 1-Jan-24, Jan-24
    return bool(re.match(r'^(\d{1,2}[-/.]\d{1,2}[-/.]\d{2,4})$', tok)) \
        or bool(re.match(r'^\d{1,2}-[A-Za-z]{3,}-\d{2,4}$', tok)) \
        or bool(re.match(r'^[A-Za-z]{3,}-\d{2,4}$', tok))

def is_number(tok):
    try:
        float(tok.replace(',', ''))
        return True
    except:
        return False

def merge_multiword_states(text):
    """Rejoin known multi-word state names to single tokens."""
    replacements = [
        ("Andaman and Nicobar Islands", "Andaman_Nicobar_Islands"),
        ("Dadra and Nagar Haveli and Daman and Diu", "Dadra_Nagar_Haveli_Daman_Diu"),
        ("Jammu and Kashmir", "Jammu_Kashmir"),
        ("Madhya Pradesh", "Madhya_Pradesh"),
        ("Tamil Nadu", "Tamil_Nadu"),
        ("Uttar Pradesh", "Uttar_Pradesh"),
        ("West Bengal", "West_Bengal"),
        ("Andhra Pradesh", "Andhra_Pradesh"),
    ]
    for a,b in replacements:
        text = text.replace(a, b)
    return text

# --- Config ---
pdf_path = "pre-monsoon_2014-2024.pdf"
good_csv = "good_data_v6_pre3.csv"

STATES = {
    'Andhra_Pradesh','Arunachal Pradesh','Assam','Bihar','Chhattisgarh','Goa',
    'Gujarat','Haryana','Himachal Pradesh','Jharkhand','Karnataka','Kerala',
    'Madhya_Pradesh','Maharashtra','Manipur','Meghalaya','Mizoram','Nagaland',
    'Odisha','Punjab','Rajasthan','Sikkim','Tamil_Nadu','Telangana','Tripura',
    'Uttar_Pradesh','Uttarakhand','West_Bengal',
    'Andaman_Nicobar_Islands','Chandigarh',
    'Dadra_Nagar_Haveli_Daman_Diu','Delhi',
    'Jammu_Kashmir','Ladakh','Lakshadweep','Puducherry'
}

# --- 1. Tokenize ---
doc = fitz.open(pdf_path)
tokens = []
for i in range(doc.page_count):
    text = doc.load_page(i).get_text("text")
    text = merge_multiword_states(text)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    tokens += text.split()

print(f"Total tokens: {len(tokens):,}")

# --- 2. Scan tokens with sliding window ---
rows = []
bad = 0
current = []

for idx, tok in enumerate(tokens):
    if tok in STATES:
        # flush old row if valid
        if len(current) == 8:
            rows.append(current)
        current = [tok]
        continue

    if not current:
        continue

    current.append(tok)

    # When buffer is between 8–12 tokens, test every 8-token slice
    if len(current) >= 8:
        found = False
        for start in range(max(0, len(current)-12), len(current)-7):
            slice8 = current[start:start+8]
            lat, lon, date, wl = slice8[4:8]
            if is_number(lat) and is_number(lon) and is_date(date) and is_number(wl):
                rows.append(slice8)
                found = True
                current = []
                break
        if not found and len(current) > 12:
            bad += 1
            current = []

# --- 3. Save ---
df = pd.DataFrame(rows, columns=[
    'STATE_UT','DISTRICT','BLOCK','VILLAGE','LATITUDE','LONGITUDE','Date','WL(mbgl)'
])
print(f"\nGood rows: {len(df):,} | Bad fragments: {bad:,}")
df.to_csv(good_csv, index=False)
print(f"✅ Saved clean data to {good_csv}")


Total tokens: 1,764,890

Good rows: 192,839 | Bad fragments: 47
✅ Saved clean data to good_data_v6_pre3.csv
